In [1]:
run "__init__.py"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from joblib import load, dump
import json
# from decouple import config
from langdetect import detect
from datetime_util import timestamp2datetime
from finnhub_api import FinnHub_init, Finnhub
from nlp import NLP_stat
from scrapy import Scrapy

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/boula/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/boula/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/boula/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/boula/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
finnhub_key=load('./finnhub/finnhub_key.pkl')

In [4]:
aapl = Finnhub(finnhub_key, "2021-06-01", "2021-06-07", "AAPL")

In [5]:
news = aapl.company_news()

In [6]:
df_news = pd.DataFrame(news)

In [35]:
df_news.head(3)

,category,datetime,headline,id,image,related,source,summary,url
0,company,1623110079,Apple's new 'private relay' feature will not b...,68011194,NaN,AAPL,Reuters,"Apple Inc on Monday said a new ""private relay""...",https://finnhub.io/api/news?id=b5c3de85e4357a4...
1,company,1623107375,"Apple's new privacy feature, designed to mask ...",68032499,https://image.cnbcfm.com/api/v1/image/10658790...,AAPL,CNBC,"Apple's new ""Private Relay"" online browsing fe...",https://finnhub.io/api/news?id=38d729b552df4d1...
2,company,1623099351,"Apple adds virtual IDs on iPhone, video plans ...",68011199,https://static.reuters.com/resources/r/?m=02&d...,AAPL,Reuters,"Apple Inc, facing accusations from rivals that...",https://finnhub.io/api/news?id=e6293a6b98321c7...


In [33]:
df_news.to_csv('./data/finnhub_news_01jun21-07jun21.csv', index=False)

In [76]:
df_news = pd.read_csv('./data/finnhub_news_01jun21-07jun21.csv')

In [4]:
scrap = Scrapy()

In [5]:
articles = scrap.scrap(df_news['url'].to_list())

In [37]:
print('Total scraped articles:', len(articles))

Total scraped articles: 133


In [38]:
np.all(df_news.shape[0] == len(articles))

True

In [39]:
dump(articles, './data/finnhub_articles_01jun21-07jun21')

['./data/finnhub_articles_01jun21-07jun21']

In [77]:
# prepare our data concatenating the 3 main dataframe for scrubbing
df_news = pd.concat([df_news, pd.Series(articles, name='articles')],
                   axis=1,
                   ignore_index=False,
                   sort=True)

In [78]:
df_news.head()

,category,datetime,headline,id,image,related,source,summary,url,articles
0,company,1623110079,Apple's new 'private relay' feature will not b...,68011194,NaN,AAPL,Reuters,"Apple Inc on Monday said a new ""private relay""...",https://finnhub.io/api/news?id=b5c3de85e4357a4...,"Discover Thomson Reuters By Stephen Nellis, Pa..."
1,company,1623107375,"Apple's new privacy feature, designed to mask ...",68032499,https://image.cnbcfm.com/api/v1/image/10658790...,AAPL,CNBC,"Apple's new ""Private Relay"" online browsing fe...",https://finnhub.io/api/news?id=38d729b552df4d1...,"In this article GUANGZHOU, China — Apple's ne..."
2,company,1623099351,"Apple adds virtual IDs on iPhone, video plans ...",68011199,https://static.reuters.com/resources/r/?m=02&d...,AAPL,Reuters,"Apple Inc, facing accusations from rivals that...",https://finnhub.io/api/news?id=e6293a6b98321c7...,Apple CEO Tim Cook greets developers during Ap...
3,company,1623099351,Factbox: Top takeaways from Apple's Developers...,68011182,https://static.reuters.com/resources/r/?m=02&d...,AAPL,Reuters,Apple Inc on Monday announced a host of new fe...,https://finnhub.io/api/news?id=b0c1f554acc6bb1...,Apple Inc (AAPL.O) on Monday announced a host ...
4,company,1623091979,Apple is turning privacy into a business advan...,68005525,https://image.cnbcfm.com/api/v1/image/10689338...,AAPL,CNBC,Privacy was mentioned as part of nearly every ...,https://finnhub.io/api/news?id=51f9e806092681a...,In this article Apple unveiled new versions o...


In [79]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133 entries, 0 to 132
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  133 non-null    object
 1   datetime  133 non-null    int64 
 2   headline  133 non-null    object
 3   id        133 non-null    int64 
 4   image     101 non-null    object
 5   related   133 non-null    object
 6   source    133 non-null    object
 7   summary   133 non-null    object
 8   url       133 non-null    object
 9   articles  80 non-null     object
dtypes: int64(2), object(8)
memory usage: 11.4+ KB


In [80]:
df_news.rename(columns={'datetime':'date'}, inplace=True)

In [81]:
# convert unix timestamp to datetime object
df_news['date'] = df_news['date'].map(lambda x: timestamp2datetime(x))

In [82]:
df_news['date'] = pd.to_datetime(df_news['date'], format='%Y-%m-%d %H:%M:%S')

In [83]:
# we got just 53 missing scraped articles, those will be droped
# drop rows with condition on columns. we're droping only row by index for every Null value on TEXT column.
df_news.drop(df_news.loc[(df_news["articles"].isna())].index, inplace=True)

In [87]:
df_news['word_count'] = df_news['articles'].apply(lambda x: len(x.split()) if type(x)!=float else x)

In [89]:
# drop only rows with 0 count
df_news.drop(df_news[(df_news['word_count']==0)].index, inplace=True)

In [91]:
df_news.head()

,category,date,headline,id,image,related,source,summary,url,articles,word_count
0,company,2021-06-07 19:54:39,Apple's new 'private relay' feature will not b...,68011194,NaN,AAPL,Reuters,"Apple Inc on Monday said a new ""private relay""...",https://finnhub.io/api/news?id=b5c3de85e4357a4...,"Discover Thomson Reuters By Stephen Nellis, Pa...",474
1,company,2021-06-07 19:09:35,"Apple's new privacy feature, designed to mask ...",68032499,https://image.cnbcfm.com/api/v1/image/10658790...,AAPL,CNBC,"Apple's new ""Private Relay"" online browsing fe...",https://finnhub.io/api/news?id=38d729b552df4d1...,"In this article GUANGZHOU, China — Apple's ne...",325
2,company,2021-06-07 16:55:51,"Apple adds virtual IDs on iPhone, video plans ...",68011199,https://static.reuters.com/resources/r/?m=02&d...,AAPL,Reuters,"Apple Inc, facing accusations from rivals that...",https://finnhub.io/api/news?id=e6293a6b98321c7...,Apple CEO Tim Cook greets developers during Ap...,840
3,company,2021-06-07 16:55:51,Factbox: Top takeaways from Apple's Developers...,68011182,https://static.reuters.com/resources/r/?m=02&d...,AAPL,Reuters,Apple Inc on Monday announced a host of new fe...,https://finnhub.io/api/news?id=b0c1f554acc6bb1...,Apple Inc (AAPL.O) on Monday announced a host ...,735
4,company,2021-06-07 14:52:59,Apple is turning privacy into a business advan...,68005525,https://image.cnbcfm.com/api/v1/image/10689338...,AAPL,CNBC,Privacy was mentioned as part of nearly every ...,https://finnhub.io/api/news?id=51f9e806092681a...,In this article Apple unveiled new versions o...,587
